## 0. Libarary 불러오기 및 경로설정

In [1]:
import torch
from sklearn.metrics import confusion_matrix, recall_score, f1_score
from tqdm import tqdm
from importlib import import_module
import multiprocessing
from dataset import JayAugmentation

/data/ephemeral/home/jae-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/data/ephemeral/home/data/train/images'

In [3]:
from dataset import MaskModelDataset

dataset = MaskModelDataset(data_dir)
class MyArg:
    resize = [128, 96]

transform = JayAugmentation(MyArg(), dataset)
dataset.set_transform(transform)

mask_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=20,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=True,
        pin_memory=torch.cuda.is_available(),
        drop_last=False,
    )

In [4]:
from models.mask_model import JayMaskModel

model = JayMaskModel().cuda()

model.eval()

labels = []
preds = []
for epoch in tqdm(range(1)):
    for index, batch in enumerate(mask_loader):
        image, label = batch
        image = image.cuda()
        label = label.long().cuda()
        labels.extend(label.cpu().numpy())
        pred = model(image)
        preds.extend(pred.cpu().numpy())
        if index > 20:
            break

/data/ephemeral/home/jae-env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/data/ephemeral/home/jae-env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1/1 [00:05<00:00,  5.24s/it]


In [5]:
recall = recall_score(labels, preds, average='macro')
f1 = f1_score(labels, preds, average='macro')
conf_matrix = confusion_matrix(labels, preds)

print("Recall:", recall)
print("f1:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Recall: 0.9548784548784549
f1: 0.924363093897032
Confusion Matrix:
[[290  18   0]
 [  0  60   5]
 [  0   0  67]]


In [11]:
torch.save(model.state_dict(), './jay/mask_model_best.pth')